In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import json
import evaluate
import tqdm

/data/katie_kang/miniconda3/envs/trlx2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-06-24 23:27:30,112] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
with open('../MATH_aug/AugMATH_part1.jsonl', 'r') as json_file:
    json_list = list(json_file)
with open('../MATH_aug/AugMATH_part2.jsonl', 'r') as json_file:
    json_list += list(json_file)
    
train_questions = []
train_answers = []
for json_str in json_list:
    result = json.loads(json_str)
    train_questions.append(result["query"])
    train_answers.append(result["response"])
train_questions = np.array(train_questions)
train_answers = np.array(train_answers)

In [35]:
full_train_part1_accs = (np.load("../MATH_aug/train_part1_answer_types5.npy")==0).sum(axis=-1)
full_train_part2_accs = (np.load("../MATH_aug/train_part2_answer_types5.npy")==0).sum(axis=-1)
full_train_accs = np.concatenate([full_train_part1_accs, full_train_part2_accs])

hard_idxs = np.where((full_train_accs>0)*(full_train_accs<3))[0]

In [36]:
ckpt = "../ckpts/math_aug_fft_rand10000/"

subsample_idxs = np.load(ckpt + "subsample_idxs.npy")[:1000]

train_samples = np.load(ckpt + "train_subsample_answers5.npy")
train_accs = (np.load(ckpt + "train_subsample_answer_types5.npy")==0)


In [37]:
hard_train_samples = []
hard_train_accs = []

true_hard_train_samples = []
ground_truth_answers = []

for i in range((1000)):
    if subsample_idxs[i] in hard_idxs:
        hard_train_samples.append(train_samples[i])
        hard_train_accs.append(train_accs[i])
        for j in range(5):
            if train_accs[i][j] == 1:
                true_hard_train_samples.append(train_samples[i][j])
                ground_truth_answers.append(train_answers[subsample_idxs[i]])

hard_train_samples = np.array(hard_train_samples)
hard_train_accs = np.array(hard_train_accs)

In [38]:
hard_train_accs.mean()

0.1003921568627451

In [25]:
true_hard_train_samples[3]

" The three-digit integer $NPM$ can be written as $100N + 10P + M$.\nWhen we divide this by $M$, we get a quotient of $100N + 10P$.\nThe remainder is $M$.\nWe are given that the quotient is the two-digit integer $MM$, so we have $100N + 10P = MM$.\nWe are also given that the digits in the hundreds place of $NPM$ are equal, so $N = P$.\nLet's call the value of $N$ (which is also the value of $P$) as $x$.\nThen the value of $NPM$ is $10x + M$.\nWe want to find the value of $M$ when $N = P = x$.\nWhen we substitute $x$ into the equation $10x + M$, we get $10x + M$.\nWe know that when we divide this by $M$, we get a quotient of $10x$.\nSo we can set up the equation $10x = M$.\nWe are given that the value of $M$ is 20, so we have $10x = 20$.\nSolving for $x$, we get $x = \\frac{20}{10} = 2$.\nTherefore, the value of $M$ is 2. The answer is: $2$"

In [26]:
ground_truth_answers[3]

' The three-digit integer $NPM$ can be written as $100N + 10P + M$.\nWhen we divide $100N + 10P + M$ by $M$, we get $MM$, or $10M + M = 11M$.\nSo we have the equation $\\frac{100N + 10P + M}{M} = 11M$.\nMultiplying both sides by $M$, we get $100N + 10P + M = 11M^2$.\nSince $N$, $P$, and $M$ are all digits, $N$ and $P$ must be between 0 and 9, and $M$ must be between 1 and 9.\nWe can try different values of $M$ to see which one satisfies the equation.\nIf we try $M = 1$, we get $100N + 10P + 1 = 11$, which is not possible since the left side is clearly larger than the right side.\nIf we try $M = 2$, we get $100N + 10P + 2 = 44$, which simplifies to $100N + 10P = 42$.\nSince $N$ and $P$ are digits, the only solution is $N = 4$, $P = 2$.\nSo $M = 2$ is the value that satisfies the equation.\nThe value of $M$ is 2. The answer is: $2$'